<a href="https://colab.research.google.com/github/nrajmalwar/END2.0/blob/main/Session_04/IMDb_Classification_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import spacy
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Create Input and target variables
# Tokenize using spacy
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm', include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
# Split the IMDB  dataset into train and test
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
# Print an example from the train data
print(vars(train_data.examples[0]))

{'text': ['Before', 'Cujo', ',', 'there', 'was', 'Lucky', 'the', 'devil', 'dog', '.', 'In', '1978,on', 'Halloween', 'night', 'the', 'movie"Devil', 'Dog', ',', 'The', 'Hound', 'of', 'Hell', '"', 'premiered', '.', 'A', 'story', 'of', 'a', 'family', 'getting', 'a', 'new', 'puppy', '(', 'from', 'a', 'farmer', 'who', 'just', 'happen', 'to', 'be', 'in', 'the', 'neighborhood', 'selling', 'fruits', 'and', 'vegetables', ')', 'because', 'their', 'dog', 'Skipper', 'was', 'killed', '.', 'Coencidence', '?', 'Everyone', 'loves', 'the', 'new', 'dog', ',', 'but', 'there', 'is', 'something', 'strange', 'about', 'him', '.', '<', 'br', '/><br', '/>It', 'is', "n't", 'long', 'until', 'the', 'father', 'Mike', 'Barry(Richard', 'Crenna', ',', 'First', 'Blood)starts', 'to', 'notice', '.', 'His', 'wife', 'Betty(Yvette', 'Mimieux', ',', 'Where', 'The', 'Boys', 'Are', ',', 'Jackson', 'County', 'Jail', ',', 'Snowbeast)is', 'different', 'and', 'his', 'kids', 'Charlie', 'and', 'Bonnie(Ike', 'Eisenman', ',', 'Witch',

In [5]:
import random

# Create validation data from train set
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
# Define the maximum number of words to be used in the vocabulary. Rest will be unknown words.
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
# Print the 20 most common words
print(TEXT.vocab.freqs.most_common(20))

[('the', 203063), (',', 192343), ('.', 166129), ('and', 109794), ('a', 109526), ('of', 100808), ('to', 93971), ('is', 76428), ('in', 61581), ('I', 54306), ('it', 53609), ('that', 49177), ('"', 44610), ("'s", 43276), ('this', 42373), ('-', 36806), ('/><br', 35659), ('was', 35096), ('as', 30590), ('with', 30113)]


In [10]:
# Print the words of first 10 tokens in the vocabulary
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [11]:
# Print labels
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [12]:
BATCH_SIZE = 64

# use cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create iterator object for train, valid and test data
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort_key=lambda x: len(x.text),
    batch_size = BATCH_SIZE,
    sort_within_batch=True,
    sort=True,
    device = device)

In [13]:
# Define the Network
class classifier(nn.Module):

    # Define all the layers used in model
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        # Embedding layer
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=False)

        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        # Use pack_padded_sequence to create equally padded sequences
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths = text_lengths.cpu(), batch_first=False, enforce_sorted=True)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        dense_outputs = self.fc(hidden[-1])

        return dense_outputs

In [14]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

# Define the network with the given parameters
model = classifier(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,867,049 trainable parameters


In [16]:
import torch.optim as optim

# Use Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [17]:
criterion = nn.BCEWithLogitsLoss()

In [18]:
# Train the model on GPU
model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # Retrieve text and no. of words
        text, text_length = batch.text

        predictions = model(text, text_length).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
# Evaluate the model on validation set
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            # Retrieve text and no. of words
            text, text_length = batch.text

            predictions = model(text, text_length).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 10

best_valid_loss = float('inf')

# Model training
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.683 | Train Acc: 55.94%
	 Val. Loss: 0.668 |  Val. Acc: 58.88%
Epoch: 02 | Epoch Time: 0m 25s
	Train Loss: 0.600 | Train Acc: 67.59%
	 Val. Loss: 0.546 |  Val. Acc: 73.60%
Epoch: 03 | Epoch Time: 0m 25s
	Train Loss: 0.485 | Train Acc: 77.50%
	 Val. Loss: 0.505 |  Val. Acc: 75.64%
Epoch: 04 | Epoch Time: 0m 25s
	Train Loss: 0.378 | Train Acc: 83.94%
	 Val. Loss: 0.619 |  Val. Acc: 71.74%
Epoch: 05 | Epoch Time: 0m 25s
	Train Loss: 0.305 | Train Acc: 87.96%
	 Val. Loss: 0.440 |  Val. Acc: 82.72%
Epoch: 06 | Epoch Time: 0m 25s
	Train Loss: 0.247 | Train Acc: 90.44%
	 Val. Loss: 0.415 |  Val. Acc: 83.17%
Epoch: 07 | Epoch Time: 0m 25s
	Train Loss: 0.221 | Train Acc: 92.07%
	 Val. Loss: 0.423 |  Val. Acc: 84.63%
Epoch: 08 | Epoch Time: 0m 25s
	Train Loss: 0.153 | Train Acc: 94.70%
	 Val. Loss: 0.457 |  Val. Acc: 84.04%
Epoch: 09 | Epoch Time: 0m 25s
	Train Loss: 0.140 | Train Acc: 95.24%
	 Val. Loss: 0.454 |  Val. Acc: 83.07%
Epoch: 10 | Epoch T

In [24]:
# Model testing
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.408 | Test Acc: 83.56%
